In [1]:
import numpy as np
from scipy import spatial 
import faiss
from time import time
import matplotlib.pyplot as plt
from collections import defaultdict

## Helper Function

In [2]:
def semi_optimized_exhaustive_search(
        index_vectors: np.ndarray,
        query_vectors: np.ndarray,
        k: int,
):
    """
    This function performs an optimized exhaustive search.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        query_vectors: An array of shape (n_queries, dim) containing the query vectors. 
        dim: The dimensionality of the vectors.
    Returns:
        An array of shape (n_queries, k) containing the indices of the k nearest neighbors for each query vector.
    """
    ann_lists = []
    for query_vec in query_vectors:
        distances = np.linalg.norm(index_vectors - query_vec, axis=1)
        ann_lists.append(list(np.argsort(distances)[:k]))
    return np.array(ann_lists)

In [4]:
def build_faiss_flatl2_index(
        index_vectors: np.ndarray,
        dim: int,
):
    """
    This function builds a Faiss flat L2 index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors. 
    Returns:
        A Faiss flat L2 index.
    """
    index = faiss.IndexFlatL2(dim)
    index.add(index_vectors)
    return index

In [5]:
def faiss_search(
        query_vectors: np.ndarray,
        index: faiss.Index,
        k: int,
):
    """
    This function uses a Faiss index to search for the k-nearest neighbors of query_vectors.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors. 
        index: A Faiss index.
        k: The number of nearest neighbors to retrieve.
    Returns:
        An array of shape (, ) containing the indices of the k-nearest neighbors for each query vector.
    """
    distances, indices = index.search(query_vectors, k)
    return indices

In [6]:
def build_faiss_lsh_index(
        index_vectors: np.ndarray,
        dim: int,
        nbits: int,
):
    """
    This function builds a Faiss LSH index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors. 
        nbits: The number of bits to use in the hash.
    Returns:
        A Faiss LSH index.
    """
    index = faiss.IndexLSH(dim, nbits)
    index.add(index_vectors)
    return index

In [7]:
def compute_recall_at_k(
        nn_gt: np.ndarray,
        ann: np.ndarray,
        k: int,
):
    """
    This function computes the recall@k.
    Args:
        nn_gt: The ground truth nearest neighbors.
        ann: The approximate nearest neighbors.
        k: The number of nearest neighbors to consider.
    Returns:
        The recall@k.
    """
    return round(sum([len(set(ann[i]) & set(nn_gt[i])) / k for i in range(len(ann))])/len(ann), 3)

# 2.1 -- LSH vs Naive Exhaustive Search (Regular Index Vectors)
### You just have to run the following cells and add the following results to the report:
* running time of the ground truth computation with semi_optimized_exhaustive_search (wall time)
* running time of creating faiss_lsh_index (wall time)
* running time of faiss_search over query_vectors with faiss_lsh_index (wall time)
* recall@10 for faiss_lsh_ann

In [8]:
query_vectors = np.load('data/query_vectors.npy')
index_vectors = np.load('data/index_vectors.npy')
k=10
dim = index_vectors.shape[1]

In [10]:
query_vectors2 = np.load('data/query_vectors2.npy')
index_vectors2 = np.load('data/index_vectors2.npy')

query_vectors3 = np.load('data/query_vectors3.npy')
index_vectors3 = np.load('data/index_vectors3.npy')

In [9]:
%%time
gt_nn = semi_optimized_exhaustive_search(index_vectors, query_vectors, k)

CPU times: total: 14.2 s
Wall time: 25.2 s


In [11]:
%%time
gt_nn2 = semi_optimized_exhaustive_search(index_vectors2, query_vectors2, k)

CPU times: total: 31.5 s
Wall time: 46.8 s


In [12]:
%%time
gt_nn3 = semi_optimized_exhaustive_search(index_vectors3, query_vectors3, k)

CPU times: total: 19.1 s
Wall time: 22.8 s


In [13]:
%%time
faiss_lsh_index = build_faiss_lsh_index(index_vectors, dim, nbits=2000)

CPU times: total: 3.58 s
Wall time: 969 ms


In [14]:
%%time
faiss_lsh_ann = faiss_search(query_vectors, faiss_lsh_index, k)

CPU times: total: 734 ms
Wall time: 122 ms


In [15]:
print(f"recall@10 for faiss_lsh_index: {compute_recall_at_k(gt_nn, faiss_lsh_ann, k)}")

recall@10 for faiss_lsh_index: 0.138


# 2.2 -- Custom Indexing Algorithm
Build an indexing algorithm that satisfies the following requirements:
* The indexing algorithm should be able to handle vectors of different dimensions
* The running time of the indexing should be less than half of the running time of semi_optimized_exhaustive_search), reported in Section 2.1.
* The running time of searching over the index should be less than a third (1/3) of the time of the semi_optimized_exhaustive_search function, reported in Section 2.1.
* The performance (in terms of recall@10) of the indexing algorithm should be at least 0.8.

The last three bullets should also appear in the report.
You are allowed to add as many helper functions as you need. You cannot use faiss of scipy libraries for this task. Numpy is allowed. 

You can also test your algorithm with the additional two query-index sets by replacing the calls made few cells ago to:

    query_vectors = np.load('data/query_vectors2.npy')
    index_vectors = np.load('data/index_vectors2.npy')
or:

    query_vectors = np.load('data/query_vectors3.npy')
    index_vectors = np.load('data/index_vectors3.npy')
    
the aforementioned requirements should also be satisfied over these two query-index sets. No need to insert the results over these two to the report.

In [148]:
import heapq

class SimpleLSH:
    def __init__(self):
        # self.num_hashes = num_hashes # some func of dim, k?
        self.hash_table = defaultdict(list)

    def generate_hash_functions(self, num_vectors, dim):
        # Generate random normal hash functions
        self.hash_functions = [np.random.randn(dim) for _ in range(12)]

    def hash(self, vec):
        return tuple([int(np.dot(h, vec) > 0) for h in self.hash_functions])
    
    def fit(self, index_vectors):
        self.index_vectors = index_vectors
        self.generate_hash_functions(*index_vectors.shape)
        for i, vec in enumerate(self.index_vectors):
            hash_key = self.hash(vec)
            self.hash_table[hash_key].append(i)
        # self.balance_hash_table()

    def euclidean_distance(self, vec1, vec2):
        # Euclidean distance implementation
        return np.linalg.norm(vec1 - vec2)
    
    def hamming_distance(self, key1, key2):
        # Hamming distance implementation
        return sum(el1 != el2 for el1, el2 in zip(key1, key2))

    # def balance_hash_table(self):
    #     small_keys = [key for key in self.hash_table.keys() if len(self.hash_table[key]) < 50]
    #     large_keys = [key for key in self.hash_table.keys() if len(self.hash_table[key]) >= 800]


    # def increase_values(self, small_keys):
    #     for hash_key in small_keys:
    #         num_vectors = len(self.hash_table[hash_key])
            
    #         # Calculate the mean of the vectors
    #         mean_vector = np.mean([self.index_vectors[i] for i in self.hash_table[hash_key]], axis=0)

    #         # Find the closest vectors from other hash keys
    #         soretd_keys = sorted([(self.hamming_distance(hash_key, other_key), other_key) 
    #                                 for other_key in self.hash_table.keys()], key=lambda x: x[0])[1:]
    #         soretd_keys = [other_key for _, other_key in soretd_keys]
    #         chosen_keys = [soretd_keys[0]]
    #         for other_key in soretd_keys:
    #             if len(self.hash_table[other_key]) + num_vectors > 50:
    #                 break
    #             else:
    #                 chosen_keys.append(other_key)
    #                 num_vectors += len(self.hash_table[other_key])

    #         candidate_vectors = []
    #         for key in chosen_keys:
    #             candidate_vectors += self.hash_table[key]
                
    #         distances = [(self.euclidean_distance(mean_vector, self.index_vectors[i]), i) for i in candidate_vectors]

    #         # Find the 10 smallest distances
    #         smallest_distances = heapq.nsmallest(50-num_vectors, distances)  

    #         # Add the indices of the 50 closest vectors to the hash table entry for the current hash key
    #         self.hash_table[hash_key].extend(index for _, index in smallest_distances)
        

    def search(self, query_vec, k):
        hash_key = self.hash(query_vec)
        if hash_key not in self.hash_table:
            return []
        # Compute distances and get the indices
        distances = [(self.euclidean_distance(query_vec, self.index_vectors[i]), i) for i in self.hash_table[hash_key]]    
        # Find the k smallest distances
        k_smallest = heapq.nsmallest(k, distances)        
        # Extract the indices of the k smallest distances
        k_smallest_indices = [index for _, index in k_smallest]
        # print(k_smallest)
        # print(k_smallest_indices)
        
        return k_smallest_indices
    
    def search_all(self, query_vectors, k):
        return [self.search(query_vec, k) for query_vec in query_vectors]

In [156]:
# import numpy as np
# import heapq
# from collections import defaultdict
# import random

# class SimpleLSH:
#     def __init__(self, num_tables=5, num_hashes=12):
#         self.num_tables = num_tables
#         self.num_hashes = num_hashes
#         self.hash_tables = [defaultdict(list) for _ in range(self.num_tables)]

#     def generate_hash_functions(self, num_vectors, dim):
#         self.hash_functions = [[np.random.randn(dim) for _ in range(self.num_hashes)] for _ in range(self.num_tables)]

#     def hash(self, vec, table_index):
#         return tuple([int(np.dot(h, vec) > 0) for h in self.hash_functions[table_index]])

#     def fit(self, index_vectors):
#         self.index_vectors = index_vectors
#         self.generate_hash_functions(*index_vectors.shape)
#         for i, vec in enumerate(self.index_vectors):
#             for table_index in range(self.num_tables):
#                 hash_key = self.hash(vec, table_index)
#                 self.hash_tables[table_index][hash_key].append(i)
#         self.balance_hash_tables()

#     def euclidean_distance(self, vec1, vec2):
#         return np.linalg.norm(vec1 - vec2)
    
#     def hamming_distance(self, key1, key2):
#         return sum(el1 != el2 for el1, el2 in zip(key1, key2))

#     def balance_hash_tables(self):
#         min_size = 10
#         max_size = 50
#         for table in self.hash_tables:
#             for hash_key in list(table.keys()):
#                 if len(table[hash_key]) > max_size:
#                     excess_indices = table[hash_key][max_size:]
#                     table[hash_key] = table[hash_key][:max_size]
#                     # Redistribute excess indices to hash keys with Hamming distance of 1
#                     for idx in excess_indices:
#                         redistributed = False
#                         for other_key in table.keys():
#                             if len(table[other_key]) < min_size and self.hamming_distance(hash_key, other_key) == 1:
#                                 table[other_key].append(idx)
#                                 redistributed = True
#                                 break
#                         if not redistributed:
#                             table[random.choice(list(table.keys()))].append(idx)

#     def search(self, query_vec, k):
#         candidates = set()
#         for table_index in range(self.num_tables):
#             hash_key = self.hash(query_vec, table_index)
#             if hash_key in self.hash_tables[table_index]:
#                 candidates.update(self.hash_tables[table_index][hash_key])
#         if not candidates:
#             return []
#         distances = [(self.euclidean_distance(query_vec, self.index_vectors[i]), i) for i in candidates]
#         k_smallest = heapq.nsmallest(k, distances)
#         k_smallest_indices = [index for _, index in k_smallest]
#         return k_smallest_indices
    
#     def search_all(self, query_vectors, k):
#         return [self.search(query_vec, k) for query_vec in query_vectors]


In [158]:
#TODO: Write your code for 2.2.2 here
# You are allowed to add more arguments to the functions and create more functions if needed.

def custom_indexing_algorithm(index_vectors, dim):
    """
    This function builds an index from scratch.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors.
    Returns:
        An index.
    """
    idx = SimpleLSH()
    idx.fit(index_vectors)
    # for key, vecs in idx.hash_table.items():
    #     print(f"Hash key: {key} has {len(vecs)} vectors")
    # print(idx.hash_functions)
    return idx


def custom_index_search(query_vectors, index, k):
    """
    This function searches over the custom index.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors.
        index: The custom index.
        k: The number of nearest neighbors to retrieve.
    """
    return index.search_all(query_vectors, k)
    

In [ ]:
# Add hyperparameters here (if needed)

In [159]:
%%time
np.random.seed(0)
custom_index = custom_indexing_algorithm(index_vectors, dim)

CPU times: total: 2.84 s
Wall time: 5.42 s


In [160]:
%%time
custom_index_ann = custom_index_search(query_vectors, custom_index, k)

CPU times: total: 14.5 s
Wall time: 29.5 s


In [ ]:
custom_index_ann

In [161]:
print(f"recall@10 for custom_index_search: {compute_recall_at_k(gt_nn2, custom_index_ann, k)}")

recall@10 for custom_index_search: 0.256
